# PINNs: a start

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import torch
import tensorflow as tf
import scipy.io
import torch.optim as optim
import time
from net import Net

## Navier-Stokes equation

Given a two-dimensional velocity field, 

$$ {\bf V}(x, y; t)  =  ( u(x, y; t), v(x, y; t) )  \; \;,  $$
its components satisfy the following equations

$$ u_t + \lambda_1 ( u u_x + v u_y ) = -p_x + \lambda_2 ( u_{xx} + u_{yy}) $$
$$ v_t + \lambda_1 ( u v_x + v v_y ) = -p_y+ \lambda_2 ( v_{xx} + v_{yy}) $$
with $ p = p(x, y; t)$ being the pressure. The parameters $\lambda_1 $ and $\lambda_2$ are unknown.

We are interested in learning the parameters $\{ \lambda \} $ as well as the pressure $ p(x, y; t)$.

Solutions are searched in a set satisfying continuity equation $ \nabla \cdot {\bf V}(x, y; t) = 0 $, 

$$ u_x + v_y = 0  \; \;.$$

Defining a latent function $ \psi = \psi(x, y; t) $ such that (how crucial is the use of $\psi$?):
$$ u = \psi_y   \;, $$ 
$$ v = - \psi_x  \;, $$ 
the continuity equation is satistied. Given a set ${\cal S}$ of (noisy) measurements of the velocity field, 

$$    {\cal S} = \{ t^{(j)}, x^{(j)}, y^{(j)} , u^{(j)}, v^{(j)}   \}_{j=1}^{N}  \;, $$
we define

$$ f(x, y; t) \equiv u_t + \lambda_1 ( u u_x + v u_y )  + p_x - \lambda_2 ( u_{xx} + u_{yy}) \;,$$    
$$  g(x, y; t) \equiv v_t + \lambda_1 ( u v_x + v v_y ) + p_y - \lambda_2 ( v_{xx} + v_{yy})  \;,$$
and proceed by jointly approximating

$$ \left[   \psi(x, y; t) ;  p(x, y; t)  \right]  \;, $$
using a single neural network with two outputs.

The prior assumption is taking into account in another neural network (PINN) with two outputs:

$$ \left[   f(x, y; t) ;  g(x, y; t)  \right]  \;. $$

The parameters $\{ \lambda \} $ operate as well as the parameters of the neural networks:


$$ \left[   \psi(x, y; t) ;  p(x, y; t)  \right]  \;, $$
$$ \left[   f(x, y; t) ;  g(x, y; t)  \right]  \;, $$
that can be trained using a mean squared error loss:

$$  MSE \equiv \frac{1}{N} \sum_{j=1}^{N} \left[  \left( u( x^{(j)}, y^{(j)}, t^{(j)}) - u^{(j)} \right)^2 + \left( v( x^{(j)}, y^{(j)}, t^{(j)}) - v^{(j)} \right)^2  \right]  + \frac{1}{N} \sum_{j=1}^{N} \left[   |  u( x^{(j)}, y^{(j)}, t^{(j)}) |^2 +  |g( x^{(j)}, y^{(j)}, t^{(j)})|^2       \right]     $$





### Data

From https://github.com/maziarraissi/PINNs/blob/master/main/Data/cylinder_nektar_wake.mat

In [2]:
data = scipy.io.loadmat('data/cylinder_nektar_wake.mat')

In [3]:
data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Fri Sep 22 23:02:15 2017',
 '__version__': '1.0',
 '__globals__': [],
 'X_star': array([[ 1.        , -2.        ],
        [ 1.07070707, -2.        ],
        [ 1.14141414, -2.        ],
        ...,
        [ 7.85858586,  2.        ],
        [ 7.92929293,  2.        ],
        [ 8.        ,  2.        ]]),
 't': array([[ 0. ],
        [ 0.1],
        [ 0.2],
        [ 0.3],
        [ 0.4],
        [ 0.5],
        [ 0.6],
        [ 0.7],
        [ 0.8],
        [ 0.9],
        [ 1. ],
        [ 1.1],
        [ 1.2],
        [ 1.3],
        [ 1.4],
        [ 1.5],
        [ 1.6],
        [ 1.7],
        [ 1.8],
        [ 1.9],
        [ 2. ],
        [ 2.1],
        [ 2.2],
        [ 2.3],
        [ 2.4],
        [ 2.5],
        [ 2.6],
        [ 2.7],
        [ 2.8],
        [ 2.9],
        [ 3. ],
        [ 3.1],
        [ 3.2],
        [ 3.3],
        [ 3.4],
        [ 3.5],
        [ 3.6],
        [ 3.7],
        [ 

In [4]:
print(data['t'].shape)
print(data['X_star'].shape)
print(data['U_star'].shape)
print(data['p_star'].shape)

(200, 1)
(5000, 2)
(5000, 2, 200)
(5000, 200)


In [5]:
U_star = data['U_star'] # N x 2 x T
p_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2

In [6]:
U_star

array([[[ 1.11419216e+00,  1.11755721e+00,  1.11956933e+00, ...,
          1.16105653e+00,  1.16209316e+00,  1.16287446e+00],
        [-4.09649775e-03,  1.09212754e-03,  3.28231641e-03, ...,
         -1.14679740e-02, -1.46226631e-02, -1.78649950e-02]],

       [[ 1.11102707e+00,  1.11424311e+00,  1.11623549e+00, ...,
          1.16119046e+00,  1.16250501e+00,  1.16355997e+00],
        [ 3.93130751e-04,  6.09231658e-03,  8.54240777e-03, ...,
         -3.69331211e-03, -6.90960992e-03, -1.02375054e-02]],

       [[ 1.10748452e+00,  1.11049848e+00,  1.11244362e+00, ...,
          1.16080365e+00,  1.16241577e+00,  1.16375737e+00],
        [ 4.42777717e-03,  1.06585027e-02,  1.33831464e-02, ...,
          4.14245483e-03,  8.86993440e-04, -2.50609725e-03]],

       ...,

       [[ 1.04183122e+00,  1.07052197e+00,  1.08570786e+00, ...,
          1.03561691e+00,  1.01482352e+00,  9.95607444e-01],
        [-1.53233747e-01, -1.52906494e-01, -1.50934775e-01, ...,
          1.70410243e-01,  1.74234

In [7]:
X_star

array([[ 1.        , -2.        ],
       [ 1.07070707, -2.        ],
       [ 1.14141414, -2.        ],
       ...,
       [ 7.85858586,  2.        ],
       [ 7.92929293,  2.        ],
       [ 8.        ,  2.        ]])

In [8]:
np.unique(X_star[:, 1])

array([-2.        , -1.91836735, -1.83673469, -1.75510204, -1.67346939,
       -1.59183673, -1.51020408, -1.42857143, -1.34693878, -1.26530612,
       -1.18367347, -1.10204082, -1.02040816, -0.93877551, -0.85714286,
       -0.7755102 , -0.69387755, -0.6122449 , -0.53061224, -0.44897959,
       -0.36734694, -0.28571429, -0.20408163, -0.12244898, -0.04081633,
        0.04081633,  0.12244898,  0.20408163,  0.28571429,  0.36734694,
        0.44897959,  0.53061224,  0.6122449 ,  0.69387755,  0.7755102 ,
        0.85714286,  0.93877551,  1.02040816,  1.10204082,  1.18367347,
        1.26530612,  1.34693878,  1.42857143,  1.51020408,  1.59183673,
        1.67346939,  1.75510204,  1.83673469,  1.91836735,  2.        ])

In [9]:
X_star.shape

(5000, 2)

In [10]:
p_star

array([[-0.10815457, -0.11115509, -0.1123679 , ..., -0.10035143,
        -0.09845376, -0.09658143],
       [-0.10560371, -0.10881921, -0.1101648 , ..., -0.10137256,
        -0.09959008, -0.09781616],
       [-0.10257633, -0.10599348, -0.10746686, ..., -0.10204929,
        -0.10039933, -0.09874095],
       ...,
       [ 0.0066028 ,  0.00272234,  0.00081532, ...,  0.01152894,
         0.01414437,  0.01651662],
       [ 0.007892  ,  0.00402109,  0.00209731, ...,  0.00927242,
         0.01200894,  0.01451151],
       [ 0.00915729,  0.00530504,  0.00337089, ...,  0.00691424,
         0.00975565,  0.01238328]])

In [11]:
N = X_star.shape[0]
T = t_star.shape[0]

In [12]:
# Rearrange Data 
XX = np.tile(X_star[:, 0:1], (1, T)) # N x T
YY = np.tile(X_star[:, 1:2], (1, T)) # N x T
TT = np.tile(t_star, (1, N)).T # N x T

In [13]:
print(XX.shape)
print(YY.shape)
print(TT.shape)

(5000, 200)
(5000, 200)
(5000, 200)


In [14]:
# Rearrange Data 
UU = U_star[:, 0, :] # N x T
VV = U_star[:, 1, :] # N x T
pp = p_star # N x T

In [15]:
print(UU.shape)
print(VV.shape)
print(pp.shape)

(5000, 200)
(5000, 200)
(5000, 200)


In [16]:
## Flattening
x = XX.flatten()[:, None] # NT x 1
y = YY.flatten()[:, None] # NT x 1
t = TT.flatten()[:, None] # NT x 1
    
u = UU.flatten()[:, None] # NT x 1
v = VV.flatten()[:, None] # NT x 1
p = pp.flatten()[:, None] # NT x 1

In [17]:
print(x.shape)
print(y.shape)
print(t.shape)
print(u.shape)
print(v.shape)
print(p.shape)

(1000000, 1)
(1000000, 1)
(1000000, 1)
(1000000, 1)
(1000000, 1)
(1000000, 1)


In [18]:
# Training Data    
N_train = 5000

idx = np.random.choice(N*T, N_train, replace=False)
x_train = torch.Tensor(x[idx,:])
y_train = torch.Tensor(y[idx,:])
t_train = torch.Tensor(t[idx,:])
u_train = torch.Tensor(u[idx,:])
v_train = torch.Tensor(v[idx,:])

In [19]:
x_train

tensor([[8.0000],
        [3.5455],
        [1.8485],
        ...,
        [6.6566],
        [2.6970],
        [2.4141]])

In [20]:
N_train = 5000

In [21]:
from pinn import PINN

N_train = 5000

model   = PINN(x_train, 
               y_train, 
               t_train, 
               u_train,
               v_train,
               layers_size= [3, 5],    # indentify from where this 3 comes from  
               out_size = 2,     # psi and p
               params_list= None)

In [23]:
print(list(model.parameters()))

[Parameter containing:
tensor([0.], requires_grad=True), Parameter containing:
tensor([0.], requires_grad=True), Parameter containing:
tensor([[ 0.8225,  0.8258, -1.2131],
        [ 0.3490, -0.0853, -0.2740],
        [-0.3159, -0.3708, -0.0325],
        [-1.9583,  0.0372, -0.5937],
        [ 0.6488, -0.5891,  0.6909]], requires_grad=True), Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True), Parameter containing:
tensor([[ 0.0088, -0.8376, -0.0334, -0.5507, -0.9448],
        [-0.3521, -1.0611,  0.4877, -0.0549, -0.4529]], requires_grad=True), Parameter containing:
tensor([0., 0.], requires_grad=True)]


In [34]:
optimizer = optim.SGD(params= model.parameters(), 
                      lr= 0.001, 
                      weight_decay= 0.0)

t0 = time.time()

epochs = 10

for epoch in range(epochs):
    
    u_hat, v_hat, p_hat, f_u, f_v = model.net(x_train, y_train, t_train)
    loss_ = model.loss(u_train, v_train, u_hat, v_hat, f_u, f_v)
    loss_print  = loss_

    optimizer.zero_grad()   # Clear gradients for the next mini-batches
    loss_.backward()         # Backpropagation, compute gradients
    optimizer.step()

    t1 = time.time()


    ### Training status
    print('Epoch %d, Loss= %.10f, Time= %.4f' % (epoch, 
                                                loss_print,
                                                t1-t0))

RuntimeError: grad can be implicitly created only for scalar outputs

In [26]:
X = torch.cat([x_train, y_train, t_train], dim=1)
print(X.shape)
X

torch.Size([5000, 3])


tensor([[ 8.0000,  1.0204, 11.5000],
        [ 3.5455, -1.0204, 12.8000],
        [ 1.8485,  1.1837,  0.1000],
        ...,
        [ 6.6566, -0.2857, 16.5000],
        [ 2.6970, -1.5102,  1.0000],
        [ 2.4141,  0.2041,  0.5000]])

In [27]:
model

PINN(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=5, bias=True)
  )
  (out): Linear(in_features=5, out_features=2, bias=True)
)

In [65]:
psi_p = model.forward(X)
psi_p

tensor([[-0.0192, -0.0883],
        [ 0.4411,  0.5885],
        [-0.3013, -1.4059],
        ...,
        [ 0.4463,  0.5065],
        [-0.9377, -1.1685],
        [-0.8117, -1.6665]], grad_fn=<AddmmBackward>)

In [66]:
psi = psi_p[:, 0:1]
p = psi_p[:, 1:2]
print(psi)

tensor([[-0.0192],
        [ 0.4411],
        [-0.3013],
        ...,
        [ 0.4463],
        [-0.9377],
        [-0.8117]], grad_fn=<SliceBackward>)


In [67]:
y_train

tensor([[ 1.0204],
        [-1.0204],
        [ 1.1837],
        ...,
        [-0.2857],
        [-1.5102],
        [ 0.2041]])

In [70]:
psi_np = psi.detach().numpy()
y_train_np = y_train.detach().numpy()

In [72]:
psi_np

array([[-0.01918721],
       [ 0.4410771 ],
       [-0.3013378 ],
       ...,
       [ 0.44626236],
       [-0.9376714 ],
       [-0.8117029 ]], dtype=float32)

In [76]:
out = tf.gradients(psi_np, y_train_np)[0]
print(out)

None


In [57]:
print(psi.shape)
print(y_train_.shape)

torch.Size([5000])
torch.Size([5000])


In [78]:
tf.concat([x_train, y_train, t_train], 1)

<tf.Tensor 'concat_1:0' shape=(5000, 3) dtype=float32>

In [81]:
psi.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

In [82]:
psi[0]

tensor([-0.0192], grad_fn=<SelectBackward>)

In [86]:
torch.autograd.grad(psi_[0], y_train_)

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [41]:
y

array([[-2.],
       [-2.],
       [-2.],
       ...,
       [ 2.],
       [ 2.],
       [ 2.]])

In [59]:
psi.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

In [60]:
x = torch.autograd.Variable(torch.Tensor([2]),requires_grad=True)
y = 5*x**4 + 3*x**3 + 7*x**2 + 9*x - 5

y.backward()
x.grad

tensor([233.])

In [61]:
y

tensor([145.], grad_fn=<SubBackward0>)

In [62]:
psi

tensor([-0.0192,  0.4411, -0.3013,  ...,  0.4463, -0.9377, -0.8117],
       grad_fn=<SelectBackward>)

In [63]:
y_train_ = torch.autograd.Variable(torch.Tensor(y_train),requires_grad=True)
psi_ = psi.clone()

psi_.backward()
y_train.grad

RuntimeError: grad can be implicitly created only for scalar outputs

In [64]:
psi

tensor([-0.0192,  0.4411, -0.3013,  ...,  0.4463, -0.9377, -0.8117],
       grad_fn=<SelectBackward>)

In [94]:
import torch.nn as nn

def basic_fun(x_cloned):
    res = []
    for i in range(len(x)):
        res.append(x_cloned[i] * x_cloned[i])
    print(res)
    #return Variable(torch.FloatTensor(res))
    return res[0]

def get_grad(inp, grad_var):
    A = basic_fun(inp)
    A.backward()
    return grad_var.grad


x = nn.Parameter(torch.FloatTensor([1, 2, 3, 4, 5]), requires_grad=True)
x_cloned = x.clone()
print(get_grad(x_cloned, x))

[tensor(1., grad_fn=<MulBackward0>), tensor(4., grad_fn=<MulBackward0>), tensor(9., grad_fn=<MulBackward0>), tensor(16., grad_fn=<MulBackward0>), tensor(25., grad_fn=<MulBackward0>)]
tensor([2., 0., 0., 0., 0.])


In [95]:
x_cloned

tensor([1., 2., 3., 4., 5.], grad_fn=<CloneBackward>)

In [96]:
x

Parameter containing:
tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [97]:
psi

tensor([[-0.0192],
        [ 0.4411],
        [-0.3013],
        ...,
        [ 0.4463],
        [-0.9377],
        [-0.8117]], grad_fn=<SliceBackward>)

In [98]:
psi

tensor([[-0.0192],
        [ 0.4411],
        [-0.3013],
        ...,
        [ 0.4463],
        [-0.9377],
        [-0.8117]], grad_fn=<SliceBackward>)

In [103]:
psi[0].item()

-0.019187211990356445

In [108]:
torch.autograd.grad(psi, x_train ,create_graph=True,retain_graph=True)[0]

RuntimeError: grad can be implicitly created only for scalar outputs

In [110]:
psi[0].backward(create_graph=True,retain_graph=True)

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [111]:
import torch
from copy import deepcopy

def get_gradient(f, x):
    """ computes gradient of tensor f with respect to tensor x """
    assert x.requires_grad

    x_shape = x.shape
    f_shape = f.shape
    f = f.view(-1)

    x_grads = []
    for f_val in f:
        if x.grad is not None:
            x.grad.data.zero_()
        f_val.backward(retain_graph=True)
        if x.grad is not None:
            x_grads.append(deepcopy(x.grad.data))
        else:
            # in case f isn't a function of x
            x_grads.append(torch.zeros(x.shape).to(x))
    output_shape = list(f_shape) + list(x_shape)
    return torch.cat((x_grads)).view(output_shape)

In [112]:
y_train

tensor([[ 1.0204],
        [-1.0204],
        [ 1.1837],
        ...,
        [-0.2857],
        [-1.5102],
        [ 0.2041]])

In [113]:
psi

tensor([[-0.0192],
        [ 0.4411],
        [-0.3013],
        ...,
        [ 0.4463],
        [-0.9377],
        [-0.8117]], grad_fn=<SliceBackward>)

In [123]:
get_gradient(psi_cp, y_train_cp)

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [119]:
y_train_

tensor([[ 1.0204],
        [-1.0204],
        [ 1.1837],
        ...,
        [-0.2857],
        [-1.5102],
        [ 0.2041]], requires_grad=True)

In [121]:
y_train_cp = y_train_.clone()

In [122]:
psi_cp = psi_.clone()